# Association Rules

In [15]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

### Preparing data set

In [16]:
customer_info = pd.read_csv("1st_clustering_results.csv", index_col=0)
customer_basket = pd.read_csv("customer_basket.csv", index_col=0)
customer_info_backup = pd.read_csv("customer_info_backup.csv", index_col=0)

In [17]:
outlier_ids = customer_info_backup.index.difference(customer_info.index)

In [18]:
print(len(outlier_ids))

3327


In [19]:
outliers_df = customer_info_backup.loc[outlier_ids].copy()
outliers_df['segmentation'] = 'Outliers'

In [20]:
customer_info_full = pd.concat([customer_info, outliers_df])

In [21]:
print(len(customer_info_full))


34060


In [22]:
customer_info_full[customer_info_full['segmentation'] != 'Outliers']

,segmentation,customer_name,customer_gender,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,...,lifetime_spend_hygiene,lifetime_spend_videogames,lifetime_spend_petfood,lifetime_total_distinct_products,percentage_of_products_bought_promotion,year_first_transaction,has_loyalty_card,latitude,longitude,customer_age
customer_id,,,,,,,,,,,,,,,,,,,,,
29930,Large Family Households,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6813,Promo-Oriented Seniors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39451,Large Family Households,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21557,Tech-Savvy Buyers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16415,Wide-Range Shoppers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629,Alfragide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24185,Alfragide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10075,Alfragide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Saving final clustering results including ouliers:

In [23]:
customer_info_full[['segmentation']].to_csv('clustering_results.csv')

In [24]:
basket_with_segment = customer_basket.merge(
    customer_info_full[['segmentation']], 
    left_on='customer_id', 
    right_index=True,
    how='left'
)

In [25]:
basket_with_segment

,list_of_goods,customer_id,segmentation
invoice_id,,,
5062209,"['megaman zero 3', 'energy bar', 'pokemon viol...",4925,Tech-Savvy Buyers
4272512,"['black beer', 'bramble', 'laptop', 'pokemon v...",19046,Tech-Savvy Buyers
7121052,"['soup', 'shallot', 'cake', 'fresh bread', 'cr...",10318,Young Promo-Sensitive
5847748,"['soup', 'cooking oil', 'cereals', 'oil', 'dog...",27283,Promo-Oriented Seniors
6336114,"['eggplant', 'mint green tea', 'oil', 'bacon',...",16072,Wide-Range Shoppers
...,...,...,...
5389466,"['frozen smoothie', 'avocado', 'dog food', 'ch...",26170,Outliers
10387390,"['cake', 'antioxydant juice', 'honey', 'toothp...",19691,Vegetarians
7103407,"['candy bars', 'spaghetti', 'oil', 'final fant...",1720,Young Promo-Sensitive


### Goods by segment

In [26]:
basket_with_segment.groupby('segmentation')['list_of_goods'].count()


segmentation
Alfragide                    274
High-Spending Shoppers     10234
Large Family Households    15888
Outliers                    9742
Promo-Oriented Seniors     14187
Tech-Savvy Buyers           9575
Vegetarians                12812
Wide-Range Shoppers        15150
Young Promo-Sensitive      12138
Name: list_of_goods, dtype: int64

In [27]:
segmenti = basket_with_segment['segmentation'].unique()
segment_rules = {}

for segment in segmenti:
    basket_seg = basket_with_segment[basket_with_segment['segmentation'] == segment]
    transactions = basket_seg["list_of_goods"].apply(eval).tolist()
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    frequent_itemsets = apriori(
        df_encoded, 
        min_support=0.01, 
        use_colnames=True, 
        max_len=2, 
        low_memory=True
    )
    
    rules = association_rules(
        frequent_itemsets, 
        metric="lift", 
        min_threshold=1.1
    ).sort_values("lift", ascending=False).reset_index(drop=True)
    
    segment_rules[segment] = rules


This code finds the strongest product associations for each customer segment, allowing you to see which products are commonly bought together within each group.

In [28]:
for segment in segment_rules:
    rules = segment_rules[segment].head(10)
    print(f"\nTop ruler for the segment {segment}")
    display(rules)



Top ruler for the segment Tech-Savvy Buyers


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(white wine),(cider),0.188198,0.147990,0.092637,0.492231,3.326119,1.0,0.064786,1.677948,0.861478,0.380360,0.404034,0.559101
1,(cider),(white wine),0.147990,0.188198,0.092637,0.625970,3.326119,1.0,0.064786,2.170420,0.820822,0.380360,0.539260,0.559101
2,(dessert wine),(white wine),0.122507,0.188198,0.070183,0.572890,3.044074,1.0,0.047127,1.900685,0.765240,0.291793,0.473874,0.472905
3,(white wine),(dessert wine),0.188198,0.122507,0.070183,0.372919,3.044074,1.0,0.047127,1.399330,0.827164,0.291793,0.285372,0.472905
4,(cider),(dessert wine),0.147990,0.122507,0.053159,0.359210,2.932167,1.0,0.035030,1.369392,0.773412,0.244594,0.269749,0.396570
5,(dessert wine),(cider),0.122507,0.147990,0.053159,0.433930,2.932167,1.0,0.035030,1.505133,0.750952,0.244594,0.335607,0.396570
6,(bramble),(dessert wine),0.084282,0.122507,0.028198,0.334572,2.731058,1.0,0.017873,1.318691,0.692180,0.157895,0.241672,0.282376
7,(dessert wine),(bramble),0.122507,0.084282,0.028198,0.230179,2.731058,1.0,0.017873,1.189521,0.722332,0.157895,0.159325,0.282376
8,(champagne),(white wine),0.090966,0.188198,0.046162,0.507463,2.696424,1.0,0.029042,1.648203,0.692096,0.198117,0.393279,0.376373
9,(white wine),(champagne),0.188198,0.090966,0.046162,0.245283,2.696424,1.0,0.029042,1.204470,0.774990,0.198117,0.169759,0.376373



Top ruler for the segment Young Promo-Sensitive


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(barbecue sauce),(deodorant),0.044488,0.037898,0.011040,0.248148,6.547874,1.0,0.009354,1.279644,0.886728,0.154734,0.218532,0.269726
1,(deodorant),(barbecue sauce),0.037898,0.044488,0.011040,0.291304,6.547874,1.0,0.009354,1.348268,0.880653,0.154734,0.258308,0.269726
2,(cologne),(barbecue sauce),0.047207,0.044488,0.011040,0.233857,5.256583,1.0,0.008940,1.247171,0.849883,0.136874,0.198185,0.241003
3,(barbecue sauce),(cologne),0.044488,0.047207,0.011040,0.248148,5.256583,1.0,0.008940,1.267261,0.847465,0.136874,0.210897,0.241003
4,(beer),(salt),0.131817,0.069781,0.042594,0.323125,4.630568,1.0,0.033395,1.374285,0.903086,0.267876,0.272349,0.466757
5,(salt),(beer),0.069781,0.131817,0.042594,0.610390,4.630568,1.0,0.033395,2.228335,0.842859,0.267876,0.551234,0.466757
6,(white wine),(beer),0.130499,0.131817,0.070275,0.538510,4.085272,1.0,0.053073,1.881260,0.868565,0.365937,0.468441,0.535818
7,(beer),(white wine),0.131817,0.130499,0.070275,0.533125,4.085272,1.0,0.053073,1.862384,0.869884,0.365937,0.463054,0.535818
8,(cider),(white wine),0.097133,0.130499,0.051162,0.526718,4.036173,1.0,0.038486,1.837171,0.833169,0.289916,0.455685,0.459382
9,(white wine),(cider),0.130499,0.097133,0.051162,0.392045,4.036173,1.0,0.038486,1.485090,0.865141,0.289916,0.326640,0.459382



Top ruler for the segment Promo-Oriented Seniors


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(milk),(cream),0.153575,0.052933,0.010008,0.065164,1.231064,1.0,0.001878,1.013083,0.221750,0.050929,0.012915,0.127112
1,(cream),(milk),0.052933,0.153575,0.010008,0.189061,1.231064,1.0,0.001878,1.043759,0.198185,0.050929,0.041924,0.127112
2,(burgers),(sparkling water),0.224635,0.036883,0.010133,0.045111,1.223069,1.0,0.001848,1.008616,0.235224,0.040310,0.008543,0.159927
3,(sparkling water),(burgers),0.036883,0.224635,0.010133,0.274744,1.223069,1.0,0.001848,1.069092,0.189369,0.040310,0.064626,0.159927
4,(cooking oil),(metroid fusion),0.357880,0.023225,0.010070,0.028139,1.211591,1.0,0.001759,1.005057,0.271972,0.027142,0.005031,0.230872
5,(metroid fusion),(cooking oil),0.023225,0.357880,0.010070,0.433604,1.211591,1.0,0.001759,1.133695,0.178791,0.027142,0.117928,0.230872
6,(escalope),(olive oil),0.200969,0.063633,0.015483,0.077044,1.210749,1.0,0.002695,1.014530,0.217846,0.062153,0.014322,0.160183
7,(olive oil),(escalope),0.063633,0.200969,0.015483,0.243323,1.210749,1.0,0.002695,1.055974,0.185894,0.062153,0.053007,0.160183
8,(milk),(pokemon violet),0.153575,0.124182,0.022910,0.149180,1.201306,1.0,0.003839,1.029382,0.197977,0.089899,0.028543,0.166835
9,(pokemon violet),(milk),0.124182,0.153575,0.022910,0.184491,1.201306,1.0,0.003839,1.037910,0.191333,0.089899,0.036525,0.166835



Top ruler for the segment High-Spending Shoppers


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(cider),(white wine),0.036545,0.043189,0.015634,0.427807,9.905389,1.0,0.014056,1.672183,0.933147,0.243902,0.401979,0.394899
1,(white wine),(cider),0.043189,0.036545,0.015634,0.361991,9.905389,1.0,0.014056,1.510096,0.939627,0.243902,0.337791,0.394899
2,(white wine),(dessert wine),0.043189,0.033027,0.012703,0.294118,8.905325,1.0,0.011276,1.369878,0.927778,0.200000,0.270008,0.339367
3,(dessert wine),(white wine),0.033027,0.043189,0.012703,0.384615,8.905325,1.0,0.011276,1.554817,0.918027,0.200000,0.356838,0.339367
4,(meatballs),(blueberries),0.057846,0.132011,0.010260,0.177365,1.343562,1.0,0.002624,1.055132,0.271410,0.057127,0.052252,0.127543
5,(blueberries),(meatballs),0.132011,0.057846,0.010260,0.077720,1.343562,1.0,0.002624,1.021549,0.294600,0.057127,0.021094,0.127543
6,(hot dogs),(laptop),0.044069,0.200410,0.011628,0.263858,1.316589,1.0,0.002796,1.086189,0.251547,0.049937,0.079350,0.160939
7,(laptop),(hot dogs),0.200410,0.044069,0.011628,0.058020,1.316589,1.0,0.002796,1.014811,0.300731,0.049937,0.014595,0.160939
8,(french wine),(shrimp),0.104749,0.074849,0.010162,0.097015,1.296150,1.0,0.002322,1.024548,0.255218,0.059977,0.023960,0.116393
9,(shrimp),(french wine),0.074849,0.104749,0.010162,0.135770,1.296150,1.0,0.002322,1.035895,0.246970,0.059977,0.034651,0.116393



Top ruler for the segment Alfragide


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(megaman zero 4),(mashed potato),0.014599,0.032847,0.010949,0.750000,22.833333,1.0,0.010469,3.868613,0.970370,0.300000,0.741509,0.541667
1,(mashed potato),(megaman zero 4),0.032847,0.014599,0.010949,0.333333,22.833333,1.0,0.010469,1.478102,0.988679,0.300000,0.323457,0.541667
2,(pickles),(pokemon violet),0.036496,0.032847,0.010949,0.300000,9.133333,1.0,0.009750,1.381648,0.924242,0.187500,0.276226,0.316667
3,(pokemon violet),(pickles),0.032847,0.036496,0.010949,0.333333,9.133333,1.0,0.009750,1.445255,0.920755,0.187500,0.308081,0.316667
4,(megaman zero 3),(mayonnaise),0.021898,0.058394,0.010949,0.500000,8.562500,1.0,0.009670,1.883212,0.902985,0.157895,0.468992,0.343750
5,(mayonnaise),(megaman zero 3),0.058394,0.021898,0.010949,0.187500,8.562500,1.0,0.009670,1.203818,0.937984,0.157895,0.169310,0.343750
6,(almonds),(flax seed),0.029197,0.047445,0.010949,0.375000,7.903846,1.0,0.009564,1.524088,0.899749,0.166667,0.343870,0.302885
7,(flax seed),(almonds),0.047445,0.029197,0.010949,0.230769,7.903846,1.0,0.009564,1.262044,0.916986,0.166667,0.207634,0.302885
8,(sparkling water),(strong cheese),0.076642,0.018248,0.010949,0.142857,7.828571,1.0,0.009550,1.145377,0.944664,0.130435,0.126925,0.371429
9,(strong cheese),(sparkling water),0.018248,0.076642,0.010949,0.600000,7.828571,1.0,0.009550,2.308394,0.888476,0.130435,0.566798,0.371429


This table shows which products are frequently bought together and how strong those associations are. Includes:
- Antecedent support: Fraction of transactions containing the antecedent item(s)
- Consequent support: Fraction of transactions containing the consequent item(s)
- Support: Fraction of transactions containing both antecedent and consequent
- The confidence will give us the probability of buying the consequent given the antecedent (support/antecedent support)
- Lift: How much more likely the consequent is bought with the antecedent, compared to random
- Conviction: How strongly the antecedent implies the consequent (higher = stronger implication)  
  
And other metrics...

It is also interesting to look at the negative association, meaning two products are bought together less often than expected by chance. This could reveal products that customers see as substitutes, and preferences between segments, which is interesting for promotion planning and store layout.

In [29]:
for segment in segment_rules:
    # Get the last 10 rules (lowest lift)
    negative_rules = segment_rules[segment].tail(10)
    print(f"\nLowest lift (potential negative association) rules for the segment {segment}")
    display(negative_rules)


Lowest lift (potential negative association) rules for the segment Tech-Savvy Buyers


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
650,(iMac),(portal 2),0.176501,0.052637,0.010235,0.057988,1.101660,1.0,0.000944,1.005680,0.112057,0.046756,0.005648,0.126216
651,(portal 2),(iMac),0.052637,0.176501,0.010235,0.194444,1.101660,1.0,0.000944,1.022274,0.097406,0.046756,0.021789,0.126216
652,(pancakes),(cream),0.470392,0.022977,0.011906,0.025311,1.101597,1.0,0.001098,1.002395,0.174141,0.024729,0.002389,0.271746
653,(cream),(pancakes),0.022977,0.470392,0.011906,0.518182,1.101597,1.0,0.001098,1.099187,0.094396,0.024729,0.090237,0.271746
654,(gadget for tiktok streaming),(portal),0.309661,0.049922,0.017023,0.054975,1.101219,1.0,0.001565,1.005347,0.133146,0.049695,0.005319,0.197989
655,(portal),(gadget for tiktok streaming),0.049922,0.309661,0.017023,0.341004,1.101219,1.0,0.001565,1.047563,0.096745,0.049695,0.045403,0.197989
656,(soda),(energy drink),0.019112,0.650235,0.013681,0.715847,1.100905,1.0,0.001254,1.230904,0.093442,0.020867,0.187589,0.368444
657,(energy drink),(soda),0.650235,0.019112,0.013681,0.021041,1.100905,1.0,0.001254,1.001970,0.262052,0.020867,0.001966,0.368444
658,(laptop),(bluetooth headphones),0.129086,0.099948,0.014204,0.110032,1.100899,1.0,0.001302,1.011331,0.105235,0.066116,0.011204,0.126072
659,(bluetooth headphones),(laptop),0.099948,0.129086,0.014204,0.142111,1.100899,1.0,0.001302,1.015182,0.101829,0.066116,0.014955,0.126072



Lowest lift (potential negative association) rules for the segment Young Promo-Sensitive


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
642,(oil),(zucchini),0.651178,0.028094,0.020185,0.030997,1.103346,1.0,0.001891,1.002996,0.268522,0.030625,0.002987,0.374736
643,(zucchini),(oil),0.028094,0.651178,0.020185,0.718475,1.103346,1.0,0.001891,1.239044,0.096374,0.030625,0.192926,0.374736
644,(oil),(green grapes),0.651178,0.027764,0.019937,0.030617,1.102772,1.0,0.001858,1.002943,0.267168,0.030254,0.002935,0.374359
645,(green grapes),(oil),0.027764,0.651178,0.019937,0.718101,1.102772,1.0,0.001858,1.237400,0.095855,0.030254,0.191854,0.374359
646,(gums),(cooking oil),0.109656,0.463091,0.055940,0.510143,1.101603,1.0,0.005159,1.096052,0.103592,0.108242,0.087634,0.315470
647,(cooking oil),(gums),0.463091,0.109656,0.055940,0.120797,1.101603,1.0,0.005159,1.012672,0.171784,0.108242,0.012514,0.315470
648,(whole weat flour),(cooking oil),0.053963,0.463091,0.027517,0.509924,1.101130,1.0,0.002527,1.095562,0.097081,0.056210,0.087226,0.284672
649,(cooking oil),(whole weat flour),0.463091,0.053963,0.027517,0.059420,1.101130,1.0,0.002527,1.005802,0.171057,0.056210,0.005769,0.284672
650,(milk),(cooking oil),0.073076,0.463091,0.037238,0.509583,1.100394,1.0,0.003397,1.094800,0.098428,0.074637,0.086591,0.294998
651,(cooking oil),(milk),0.463091,0.073076,0.037238,0.080413,1.100394,1.0,0.003397,1.007978,0.169926,0.074637,0.007915,0.294998



Lowest lift (potential negative association) rules for the segment Promo-Oriented Seniors


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
280,(eggs),(ratchet & clank),0.094914,0.568102,0.059353,0.625332,1.100739,1.0,0.005432,1.152748,0.101117,0.098321,0.132508,0.364904
281,(ratchet & clank),(eggs),0.568102,0.094914,0.059353,0.104476,1.100739,1.0,0.005432,1.010677,0.211900,0.098321,0.010564,0.364904
282,(hot dogs),(ratchet & clank 2),0.039527,0.256105,0.011140,0.281847,1.100513,1.0,0.001017,1.035845,0.095091,0.039159,0.034604,0.162673
283,(ratchet & clank 2),(hot dogs),0.256105,0.039527,0.011140,0.043500,1.100513,1.0,0.001017,1.004154,0.122777,0.039159,0.004136,0.162673
284,(turkey),(ratchet & clank),0.108447,0.568102,0.067787,0.625073,1.100283,1.0,0.006178,1.151952,0.102229,0.111352,0.131908,0.372197
285,(ratchet & clank),(turkey),0.568102,0.108447,0.067787,0.119322,1.100283,1.0,0.006178,1.012349,0.211028,0.111352,0.012198,0.372197
286,(ratchet & clank),(chocolate bread),0.568102,0.018630,0.011644,0.020496,1.100155,1.0,0.001060,1.001905,0.210784,0.020247,0.001901,0.322748
287,(chocolate bread),(ratchet & clank),0.018630,0.568102,0.011644,0.625000,1.100155,1.0,0.001060,1.151729,0.092766,0.020247,0.131740,0.322748
288,(grated cheese),(pet food),0.045569,0.298842,0.014980,0.328729,1.100011,1.0,0.001362,1.044524,0.095259,0.045472,0.042626,0.189428
289,(pet food),(grated cheese),0.298842,0.045569,0.014980,0.050126,1.100011,1.0,0.001362,1.004798,0.129668,0.045472,0.004775,0.189428



Lowest lift (potential negative association) rules for the segment High-Spending Shoppers


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
486,(frozen smoothie),(final fantasy XXII),0.294704,0.032539,0.010553,0.035809,1.100509,1.0,0.000964,1.003392,0.129491,0.033323,0.003380,0.180067
487,(final fantasy XXII),(frozen smoothie),0.032539,0.294704,0.010553,0.324324,1.100509,1.0,0.000964,1.043838,0.094401,0.033323,0.041997,0.180067
488,(vacuum cleaner),(avocado),0.043678,0.254153,0.012214,0.279642,1.100291,1.0,0.001113,1.035384,0.095313,0.042764,0.034175,0.163850
489,(avocado),(vacuum cleaner),0.254153,0.043678,0.012214,0.048058,1.100291,1.0,0.001113,1.004602,0.122209,0.042764,0.004581,0.163850
490,(avocado),(ring light),0.254153,0.052765,0.014755,0.058055,1.100242,1.0,0.001344,1.005615,0.122155,0.050502,0.005584,0.168842
491,(ring light),(avocado),0.052765,0.254153,0.014755,0.279630,1.100242,1.0,0.001344,1.035366,0.096184,0.050502,0.034158,0.168842
492,(frozen smoothie),(fromage blanc),0.294704,0.146766,0.047586,0.161472,1.100204,1.0,0.004334,1.017538,0.129134,0.120814,0.017236,0.242853
493,(fromage blanc),(frozen smoothie),0.146766,0.294704,0.047586,0.324234,1.100204,1.0,0.004334,1.043699,0.106744,0.120814,0.041870,0.242853
494,(bluetooth headphones),(shower gel),0.436682,0.072210,0.034688,0.079436,1.100067,1.0,0.003155,1.007849,0.161479,0.073151,0.007788,0.279908
495,(shower gel),(bluetooth headphones),0.072210,0.436682,0.034688,0.480379,1.100067,1.0,0.003155,1.084094,0.098044,0.073151,0.077571,0.279908



Lowest lift (potential negative association) rules for the segment Alfragide


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1252,(oatmeal),(salt),0.083942,0.156934,0.014599,0.173913,1.108190,1.0,0.001425,1.020553,0.106574,0.064516,0.020139,0.133468
1253,(salt),(oatmeal),0.156934,0.083942,0.014599,0.093023,1.108190,1.0,0.001425,1.010013,0.115801,0.064516,0.009914,0.133468
1254,(beer),(meatballs),0.149635,0.551095,0.091241,0.609756,1.106445,1.0,0.008778,1.150319,0.113133,0.149701,0.130676,0.387660
1255,(meatballs),(beer),0.551095,0.149635,0.091241,0.165563,1.106445,1.0,0.008778,1.019088,0.214309,0.149701,0.018731,0.387660
1256,(salt),(mineral water),0.156934,0.463504,0.080292,0.511628,1.103827,1.0,0.007552,1.098540,0.111570,0.148649,0.089701,0.342428
1257,(mineral water),(salt),0.463504,0.156934,0.080292,0.173228,1.103827,1.0,0.007552,1.019708,0.175325,0.148649,0.019327,0.342428
1258,(salt),(seabass),0.156934,0.189781,0.032847,0.209302,1.102862,1.0,0.003064,1.024689,0.110630,0.104651,0.024094,0.191190
1259,(seabass),(salt),0.189781,0.156934,0.032847,0.173077,1.102862,1.0,0.003064,1.019521,0.115115,0.104651,0.019148,0.191190
1260,(eggs),(pasta),0.175182,0.321168,0.062044,0.354167,1.102746,1.0,0.005781,1.051095,0.112962,0.142857,0.048611,0.273674
1261,(pasta),(eggs),0.321168,0.175182,0.062044,0.193182,1.102746,1.0,0.005781,1.022309,0.137255,0.142857,0.021822,0.273674


### Targeted Promotions

**Tech-Savvy Buyers**
This group shows strong associations between premium or niche alcoholic beverages such as white wine, cider, dessert wine, champagne and bramble. Their high lift values and confidence scores suggest cross-category buying behaviors among trendy or gourmet drink options. Since these customers are likely to be active on mobile apps or digital platforms we will focus on digital, experiential, and premium promotion mechanics.  
Our promotion idea for this segment is a  **“Curated Experience Box”**:  
- Includes: 1 bottle of white wine, 1 bottle of cider, gourmet sides (e.g., olives, cheese).
- The packaging includes a QR code that when scanned with the supermarket’s app gives access to a virtual tasting experience + 10% off other alcoholic beverages.
- Delivery Channel: Mobile App + Web platform (with push notifications and gamification) and limited-time offer with countdown timers.
  
**Young Promo-Sensitive**
These shoppers are highly sensitive to deals and promotions, so it is important to give them different promotions. We found positive patterns like: 
- Barbecue Sauce + Deodorant/Cologne (Lift > 5)
- Salt + Beer / White Wine  (Lift > 4)
- Cider + White Wine
And negative association between cooking staples like oil, whole wheat flour, zucchini, green grapes, indicating "Healthy image" product bundles are not appealing to this segment.  
Their behavior indicates impulse-driven buying when promotions are present and a strong tendency to respond to combo offers, especially involving categories like personal care (deodorant, cologne) and leisure items (barbecue sauce, beer, cider). Strong crossover between lifestyle and grooming suggests a social/party persona, which encourages multi-category purchases with a fun and event-oriented framing. Negative associations show low interest in cooking basics and wellness foods (e.g., oil, zucchini, whole wheat flour), indicating they may skip health staples in favor of indulgent or trendy purchases.   

Our promotion idea for this segment is a **“Grill Essentials Kit”**:
- The bundle includes 1 BBQ Sauce, salt and a pack of beers.
- Near the “Grill Essentials Kit” we will place a deodorant + cologne bundle that will be 30% off when bought together with the grill bundle.
- Delivery channel: App + in-store endcap displays

For this group, the  “Curated Experience Box” promotion will also be interesting given their high interest and association between alcoholic beverages.  

**Promo-Oriented Seniors**
Despite being older, this segment exhibits similar behaviors to the Young Promo-Sensitive group, responding well to indulgent, social, or event-focused promotions. The “Curated Experience Box”, and “Grill Essentials Kit” will be good promotions for this group since they show positive association between alcoholic beverages and grill products like BBQ sauce, salt and beer. They also show association between pantry purchases like cereals, butter, or fresh bread, and pet food.  
Our promotion idea for this segment is a **20% discount on petfood when spending on pantry essential**:
- Spend over 10 euros on pantry essentials (bread, cereal, butter…) and get 20% off on pet food.
- Delivery Channels: In-store signage near relevant sections + optional digital coupon via app (although this group isn't especially prone to app usage).
We also propose to set a sauce stand near fries, since mayo and fries seems to have a positive association for this segment.  

**Wide-Range Shoppers**
This segment likes alcohol variety, suggesting a refined or exploratory drink preference likely social, occasion-based, or hobbyist consumption. They're less attracted to essentials like flour, oil, or odd mixes like barbecue sauce with dark chocolate. The associations between white wine, cider, dessert wine, beer and bramble show a very high lift (>8). Seeing the interest this group and others have on alcoholic beverages, we suggest another alcohol based promotion.
Our promotion idea for this segment (and other alcohol indulging segments) is a **“Buy 2, Get 1 Free Happy Hour”**:
- From 17:00 to 19:00 Monday through Friday customers can get a free bottle when buying 2 other alcoholic beverages.
- Delivery Channels: In-store posters + app push notifications during active hours.
  
**Vegetarians**
This group shows health-forward eating habits, with high association between salad, flax seed, green grapes, melons.  Notably, there are also links to personal care products (e.g., body spray, shower gel, cologne), showing a self-care mindset.  
Our promotion idea for this segment is a **“Salad Builder”** bundle:
- Salad bundle: prewashed salad mix, flax seeds and cherry tomatoes.
- Fruit salad bundle: green grapes, melons and strawberries.
- Buy one of the bundles and get the other 25% off.  
- Also provide a “Save it!” discount when bundles are near the expiration date.
- Delivery Channels: Visual-focused in-store signage + health-themed app messaging.  
Framing should emphasize nourishment, convenience, and freshness.  
  
**Large Family Households**
This segment includes large families with kids, which shows in the association rules where food staples like milk, cooking oil or burgers are associated with each other and also with games for the youngest like Pokemon Violet.  
Our promotion idea for this segment is a  **“Grocery + Game Combo”**:
- Spend €50+ on groceries and receive 20% off a videogame of your choice.
- Large families show a high spend on groceries and this will promote videogame purchases like Pokemon Violet or Metroid Fusion.
- Delivery Channels: App-exclusive promotion + targeted posters in-store (e.g., near game section or kids' products).
  
**High Spending Shoppers**
These shoppers enjoy gourmet products and often purchase together products like cider/dessert wine and white whine, meatballs and blueberries, shrimp and french whine… Judging by their premium lifestyle these buyers won’t focus as much on promotions, but will enjoy previous alcoholic beverages promotions: “Curated Experience Box” and “Buy 2, Get 1 Free Happy Hour”.
  
**Alfragide**
The Alfragide segment exhibits some of the highest lift association rules among all segments, suggesting a shopper profile that is: Quirky and niche, interested in premium tech and food and willing to mix luxury with practicality. Interestingly, the key insights we saw are:
- Magman Zero 4 → Mashed Potato (Lift = 22.83)
- Pickles → Pokémon Violet (Lift = 9.13)
- Megaman Zero 3 → Mayonnaise (Lift = 8.56)
For this segment we believe our previous “Grocery + Game Combo” would be the most beneficial while the delivery method can include posters in Alfragide store.

**Outliers**
This group reflects outliers we found in the data and the rules we find for them are interesting across all segments and stores.The rules we saw highlight non-mainstream, or unexpected item pairings with strong lift. Some main associations found include:
- Pancakes → Energy Drink
- Minecraft → Ratchet & Clank
- Babies Food → Ratchet & Clank
- Minecraft → Babies Food
Again, we can see a pattern of videogame purchases. These people would also benefit from our  “Grocery + Game Combo” previously mentioned.
